In [ ]:
import pandas as pd
import requests
import math
import numpy as np
import io
import ast
from datetime import datetime

In [ ]:
# 展開所有dataframe columns
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option ('display.max_colwidth',500)

# 不縮寫成科學記號
pd.set_option('display.float_format',lambda x : '%.2f' % x)

In [ ]:
row = pd.read_json('./tw.json')
row['code'] = row[0].str.split(pat=":").str[1] # 從TWS:020038:STOCK取出020038
row['name'] = row[200008]
row['close'] = row[6]
row

In [ ]:
df = row[['code', 'name', 'close']]
df

In [ ]:
# code to cnyesId
def getCnyesId(id):
    r = requests.get(f'https://fund.api.cnyes.com/fund/api/v1/funds/{id}/etf/mapping').json()
    return r['items']['cnyesId']

getCnyesId('020038')

In [ ]:
# df['cnyesId'] = df.apply(lambda r: getCnyesId(r['code']), axis=1)

In [ ]:
# df.to_csv('etf.csv', index = False)

## ETF成分股

In [ ]:
def p(id):
    if(id):
        print(id)
    else:
        print(None)
    
df.apply(lambda r: p(r['cnyesId']), axis=1)

In [ ]:
# use cnyesId to get holdings
def getEtfHoldings(cnyesId):
    if(cnyesId):
        try:
            r = requests.get(f'https://fund.api.cnyes.com/fund/api/v1/funds/{cnyesId}/holdings').json()
            result = [[item['name'], round(item['value'], 2)] for item in r['items']['data']]
            return [item for sublist in result for item in sublist]
        except:
            return None
    else:
        return None

# getEtfHoldings('A13012')
# a = getEtfHoldings('A19114')
df['holdings'] = df.apply(lambda r: getEtfHoldings(r['cnyesId']), axis=1)

In [ ]:
df

In [ ]:
# df.to_csv('etf_holdings.csv', index = False)

In [ ]:
df = pd.read_csv('etf_holdings.csv')

In [ ]:
df.tail()

In [ ]:
holdings_columns = ['1','1(%)','2','2(%)','3','3(%)','4','4(%)','5','5(%)','6','6(%)','7','7(%)','8','8(%)','9','9(%)','10','10(%)']
df1 = pd.DataFrame(df['holdings'].fillna('[]').apply(ast.literal_eval).tolist(), columns = holdings_columns)
df1

In [ ]:
df2 = pd.concat([df, df1], axis=1) 

In [ ]:
df2 = df2[df2['holdings'].notnull()]
df2

## 基金基本資料

In [ ]:
# use cnyesId to get etf detail
def getEtfDetails(cnyesId):
    if(cnyesId):
        try:
            r = requests.get(f'https://fund.api.cnyes.com/fund/api/v1/funds/{cnyesId}/detail').json()
            netAssetInfo =  r['items']['netAssetInfo']
            return [netAssetInfo['currency'], netAssetInfo['amount'], netAssetInfo['date']]
        except:
            return None
    else:
        return None

df2['detail'] = df2.apply(lambda r: getEtfDetails(r['cnyesId']), axis=1)

In [ ]:
df2

In [ ]:
columns = ['currency', '規模(億)', '更新日期']
df_detail = pd.DataFrame(df2['detail'].tolist(), columns = columns, index=df2.index)

In [ ]:
df3 = pd.concat([df2, df_detail], axis=1)

In [ ]:
df3['規模(億)'] = round(df3['規模(億)']/1e8, 2)

In [ ]:
df3['更新日期'] = df3['更新日期'].apply(datetime.fromtimestamp)

In [ ]:
df3['code'] = "'" + df3['code']

In [ ]:
df3

## Results

In [ ]:
df3[['code','name','規模(億)','更新日期']+holdings_columns].to_csv('etf_holdings_final.csv', index = False)